In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
!pip install emoji
import emoji
!pip install contractions
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from scipy.sparse import save_npz, load_npz
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 8.5 MB/s eta 0:00:00


In [2]:
# Download the punkt data if not already present
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Lets load the dataset and do some preliminary Analysis

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sentiment Analysis/training.1600000.processed.noemoticon.csv',encoding='latin-1',
                   names=['polarity','id','date','query','user','text'])
data.head()

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
data.shape

(1600000, 6)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   polarity  1600000 non-null  int64 
 1   id        1600000 non-null  int64 
 2   date      1600000 non-null  object
 3   query     1600000 non-null  object
 4   user      1600000 non-null  object
 5   text      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [6]:
data.isnull().sum()

,0
polarity,0
id,0
date,0
query,0
user,0
text,0


In [7]:
#Lets review class imbalance
imbalance = data['polarity'].value_counts()
imbalance

,count
polarity,
0,800000
4,800000


In [8]:
data.duplicated().sum()

np.int64(0)

No Null values or duplicated values, Also the classes positive and negative are perfectly balanced. So we are good to go ahead and start manipulating the dataset for our needs

# Lets manipulate the Data now

In [9]:
#Remove the useless columns and keep only the useful ones, that are polarity and text.
data = data[['polarity','text']]
data.sample(5)

,polarity,text
1059575,4,Voting for @mileycyrus for the MTV Movie Awards
249683,0,@borret Still 35% according to my analytics. T...
1544348,4,@supricky06 NO! I would rather eat off my leg
640749,0,Oh god there are too many films out at the mom...
978355,4,I ? YouTube.


In [10]:
#Lets de-emojitize the emojis as they carry sentiment
def convert_emoji_to_text(data):
  return emoji.demojize(str(data), delimiters=(' ',' '))

data['emoji_text'] = data['text'].apply(convert_emoji_to_text)
data.sample(10)

,polarity,text,emoji_text
662637,0,@Rand_Raspberry ouchhh that would hurt! Haha ...,@Rand_Raspberry ouchhh that would hurt! Haha ...
812103,4,@danamlewis Your cynicism won't stop me from e...,@danamlewis Your cynicism won't stop me from e...
872872,4,@GavinDeGraw yeah Gavin! Good to have you on,@GavinDeGraw yeah Gavin! Good to have you on
1005406,4,Officially a customer of @LiquidWeb now.,Officially a customer of @LiquidWeb now.
314551,0,Dreading tomorrow chemistry exam :O need a ful...,Dreading tomorrow chemistry exam :O need a ful...
335099,0,am i the only one feeling fat right now smh i...,am i the only one feeling fat right now smh i...
1465340,4,@fryfan20 heyyy... how are you? couldn't stay ...,@fryfan20 heyyy... how are you? couldn't stay ...
659278,0,@PettyshkaDM nooo you're so lucky that you ha...,@PettyshkaDM nooo you're so lucky that you ha...
913499,4,@worldofhiglet I don't think it makes you seem...,@worldofhiglet I don't think it makes you seem...
1073475,4,just received a most thoughtful gift from my b...,just received a most thoughtful gift from my b...


In [11]:
#expanding contractions and making them proper words
def expand_contractions(text):
    return contractions.fix(text)
data['expanded_text'] = data['emoji_text'].apply(expand_contractions)
data.sample(10)

,polarity,text,emoji_text,expanded_text
1211218,4,@BUZZBOT_Bob hey whats up! good to see someone...,@BUZZBOT_Bob hey whats up! good to see someone...,@BUZZBOT_Bob hey what is up! good to see someo...
185418,0,Headache hasn't gone away caffeine didn't hel...,Headache hasn't gone away caffeine didn't hel...,Headache has not gone away caffeine did not h...
377629,0,apparently iPhone or Smart Phone... I just can...,apparently iPhone or Smart Phone... I just can...,apparently iPhone or Smart Phone... I just can...
1043923,4,@itstayloryall that's cute,@itstayloryall that's cute,@itstayloryall that is cute
313349,0,can't sleep so I'm gonna re-watch the Godfath...,can't sleep so I'm gonna re-watch the Godfath...,cannot sleep so I am going to re-watch the Go...
501415,0,@Shw00 You are so lucky. I want to play those ...,@Shw00 You are so lucky. I want to play those ...,@Shw00 You are so lucky. I want to play those ...
54296,0,"Spend all this money on the gymnasium, yet I n...","Spend all this money on the gymnasium, yet I n...","Spend all this money on the gymnasium, yet I n..."
942821,4,"@kaolinfire eh, them's the breaks--at least th...","@kaolinfire eh, them's the breaks--at least th...","@kaolinfire eh, them's the breaks--at least th..."
627492,0,@KarenAlloy fuck sakes you tweet tooooo much. ...,@KarenAlloy fuck sakes you tweet tooooo much. ...,@KarenAlloy fuck sakes you tweet tooooo much. ...
525882,0,Facebook for the Pope,Facebook for the Pope,Facebook for the Pope


In [12]:
#Expand Slangs
slang_dict = {
    "lol": "laughing out loud",
    "omg": "oh my god",
    "idk": "I do not know",
    "tbh": "to be honest",
    'rofl': 'rolling on floor laughing',
    'xo':'hugs and kisses',
    'XD':'Laugh',
    'hehe':'laughter',
    'aww':'cute',
    'nah':'no',
    'rip':'rest in peace',
    'wtf':'what the fuck'
}

def expand_slang(text):
    for slang, full in slang_dict.items():
        text = text.replace(slang, full)
    return text

data['expanded_text'] = data['expanded_text'].apply(expand_slang)
data.sample(10)

,polarity,text,emoji_text,expanded_text
700666,0,I wanna be with my boys,I wanna be with my boys,I want to be with my boys
1211024,4,Awww shucks. You'd have picked me up. From 2 s...,Awww shucks. You'd have picked me up. From 2 s...,Awww shucks. You would have picked me up. From...
389336,0,My friend Grant just left for LA to live,My friend Grant just left for LA to live,My friend Grant just left for LA to live
1402807,4,twittering for the first time or is it tweeti...,twittering for the first time or is it tweeti...,twittering for the first time or is it tweeti...
1369954,4,@vwlphb Glee Club of the Damned. OMG heeeeee!,@vwlphb Glee Club of the Damned. OMG heeeeee!,@vwlphb Glee Club of the Damned. OMG heeeeee!
745783,0,Wish i was going to warped tourrr,Wish i was going to warped tourrr,Wish i was going to warped tourrr
240756,0,Hello friends! what is everyone doing today? w...,Hello friends! what is everyone doing today? w...,Hello friends! what is everyone doing today? w...
1138766,4,@Darla_Marie Well that's good! Glad to hear th...,@Darla_Marie Well that's good! Glad to hear th...,@Darla_Marie Well that is good! Glad to hear t...
21526,0,Watching The Office. Is it me or is it not as ...,Watching The Office. Is it me or is it not as ...,Watching The Office. Is it me or is it not as ...
635884,0,I have a meeting from 3 - 4! That time is for...,I have a meeting from 3 - 4! That time is for...,I have a meeting from 3 - 4! That time is for...


In [13]:
#Lets do some data cleaning now, which are:-
# Removie all urls, and @mentions and hashtags. Strip the special characters and punctuations. Remove numbers and extra whitespaces.

def cleaner(text):
  text = re.sub(r'http\S+|www.\S+|https\S+', '', text) #URLs
  text = re.sub(r'@\w+','',text) #Mentions
  text = re.sub(r'#','',text) #Hashtags
  text = re.sub(r'[^a-zA-z\s]','',text) #Special characters/numbers
  text = re.sub(r'\s+', ' ', text).strip()
  text = text.lower()
  return text

data['clean text'] = data['expanded_text'].apply(cleaner)

In [14]:
data.sample(10)

,polarity,text,emoji_text,expanded_text,clean text
420864,0,@mikeschinkel RE: Sinuses .... you too eh? Wi...,@mikeschinkel RE: Sinuses .... you too eh? Wi...,@mikeschinkel RE: Sinuses .... you too eh? Wi...,re sinuses you too eh winter here in godzone a...
213638,0,@ben_gordon lol i'm still trying unsuccessfull...,@ben_gordon lol i'm still trying unsuccessfull...,@ben_gordon laughing out loud i am still tryin...,laughing out loud i am still trying unsuccessf...
603242,0,Another early morning...I'm sooooo tired!!,Another early morning...I'm sooooo tired!!,Another early morning...I am sooooo tired!!,another early morningi am sooooo tired
1506968,4,"@thesolster @kaythepal might be there too, but...","@thesolster @kaythepal might be there too, but...","@thesolster @kaythepal might be there too, but...",might be there too but we will have to see wha...
1187966,4,its nice outside! drinkn some sky and waitin f...,its nice outside! drinkn some sky and waitin f...,its nice outside! drinkn some sky and waitin f...,its nice outside drinkn some sky and waitin fo...
720092,0,misses my hubby! and can't wait till he retur...,misses my hubby! and can't wait till he retur...,misses my hubby! and cannot wait till he retu...,misses my hubby and cannot wait till he return...
267982,0,"Inviting people over to drink some coffee, don...","Inviting people over to drink some coffee, don...","Inviting people over to drink some coffee, do ...",inviting people over to drink some coffee do n...
886139,4,"@DaeAna Yeb, or like Stephen Kings &quot;The M...","@DaeAna Yeb, or like Stephen Kings &quot;The M...","@DaeAna Yeb, or like Stephen Kings &quot;The M...",yeb or like stephen kings quotthe mistquot if ...
1165285,4,"I've been beaten by the heat, retreating insid...","I've been beaten by the heat, retreating insid...","I have been beaten by the heat, retreating ins...",i have been beaten by the heat retreating insi...
67495,0,Soooo tired trying to chill with my man tonight.,Soooo tired trying to chill with my man tonight.,Soooo tired trying to chill with my man tonight.,soooo tired trying to chill with my man tonight


In [15]:
#Check for dups and nulls again and drop them
data.drop_duplicates(subset=['clean text'], inplace=True)
data.dropna(subset=['clean text'], inplace=True)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1517962 entries, 0 to 1599999
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   polarity       1517962 non-null  int64 
 1   text           1517962 non-null  object
 2   emoji_text     1517962 non-null  object
 3   expanded_text  1517962 non-null  object
 4   clean text     1517962 non-null  object
dtypes: int64(1), object(4)
memory usage: 69.5+ MB


In [17]:
#Lets convert our string of texts into a list of tokens so they are easier to work with and compare
data['tokens'] = data['clean text'].apply(word_tokenize)

In [18]:
data.sample(10)

,polarity,text,emoji_text,expanded_text,clean text,tokens
1143754,4,@TLM26 you are going to do great at the RnR. L...,@TLM26 you are going to do great at the RnR. L...,@TLM26 you are going to do great at the RnR. L...,you are going to do great at the rnr looking f...,"[you, are, going, to, do, great, at, the, rnr,..."
285707,0,@maura_shycat I would adopt one! (if it was fr...,@maura_shycat I would adopt one! (if it was fr...,@maura_shycat I would adopt one! (if it was fr...,i would adopt one if it was friendly that is sad,"[i, would, adopt, one, if, it, was, friendly, ..."
14126,0,"aw mannn, i haven't tweeted half enough today!...","aw mannn, i haven't tweeted half enough today!...","aw mannn, i have not tweeted half enough today...",aw mannn i have not tweeted half enough today ...,"[aw, mannn, i, have, not, tweeted, half, enoug..."
550596,0,takin care of my baby... him is sick,takin care of my baby... him is sick,takin care of my baby... him is sick,takin care of my baby him is sick,"[takin, care, of, my, baby, him, is, sick]"
1242279,4,Tidying up abit.. Gna go cinema wit my best fr...,Tidying up abit.. Gna go cinema wit my best fr...,Tidying up abit.. Gna go cinema wit my best fr...,tidying up abit gna go cinema wit my best frie...,"[tidying, up, abit, gna, go, cinema, wit, my, ..."
82686,0,when the hills of CALIFONIA r burin seriously...,when the hills of CALIFONIA r burin seriously...,when the hills of CALIFONIA r burin seriously...,when the hills of califonia r burin seriously ...,"[when, the, hills, of, califonia, r, burin, se..."
437026,0,"Sister graduating today. Ugh, I hate dressing ...","Sister graduating today. Ugh, I hate dressing ...","Sister graduating today. Ugh, I hate dressing ...",sister graduating today ugh i hate dressing up,"[sister, graduating, today, ugh, i, hate, dres..."
388586,0,Taking photos of my Sony AIBO ERS312 ready to ...,Taking photos of my Sony AIBO ERS312 ready to ...,Taking photos of my Sony AIBO ERS312 ready to ...,taking photos of my sony aibo ers ready to sel...,"[taking, photos, of, my, sony, aibo, ers, read..."
525125,0,Chicken n oranges 4 lunch man its not fun eati...,Chicken n oranges 4 lunch man its not fun eati...,Chicken n oranges 4 lunch man its not fun eati...,chicken n oranges lunch man its not fun eatin ...,"[chicken, n, oranges, lunch, man, its, not, fu..."
26005,0,"watching wow wow wubzy with logan, and then sa...","watching wow wow wubzy with logan, and then sa...","watching wow wow wubzy with logan, and then sa...",watching wow wow wubzy with logan and then say...,"[watching, wow, wow, wubzy, with, logan, and, ..."


In [19]:
#Lets remove some more useless stopwords like (the, us, in, and) which are useless for the analysis
custom_stopwords = ['rt', 'via', 'amp', 'u', 'im', 'ur']
stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)
stop_words.discard('not')
stop_words.discard('no')
stop_words.discard('yes')
data['tokens'] = data['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

In [20]:
data.sample(10)

,polarity,text,emoji_text,expanded_text,clean text,tokens
933507,4,is hating this picture..,is hating this picture..,is hating this picture..,is hating this picture,"[hating, picture]"
811218,4,@stablehost ah I have just about everyone I fo...,@stablehost ah I have just about everyone I fo...,@stablehost ah I have just about everyone I fo...,ah i have just about everyone i follow going t...,"[ah, everyone, follow, going, cell, phone, sms..."
537178,0,worried about Thomas being out in this nasty w...,worried about Thomas being out in this nasty w...,worried about Thomas being out in this nasty w...,worried about thomas being out in this nasty w...,"[worried, thomas, nasty, weather]"
6737,0,@jlb923 that link just takes you to the mlb sh...,@jlb923 that link just takes you to the mlb sh...,@jlb923 that link just takes you to the mlb sh...,that link just takes you to the mlb shop main ...,"[link, takes, mlb, shop, main, page]"
954820,4,"@miizronnie Hi Ronnie! I'm fine, had an awesom...","@miizronnie Hi Ronnie! I'm fine, had an awesom...","@miizronnie Hi Ronnie! I am fine, had an aweso...",hi ronnie i am fine had an awesome day yesterd...,"[hi, ronnie, fine, awesome, day, yesterday, nk..."
1203862,4,@LidiaAnain u know what....i know where it was...,@LidiaAnain u know what....i know where it was...,@LidiaAnain you know what....i know where it w...,you know whati know where it wasi was in one o...,"[know, whati, know, wasi, one, dreamsyeah, lau..."
514441,0,and she threw a phone at my face,and she threw a phone at my face,and she threw a phone at my face,and she threw a phone at my face,"[threw, phone, face]"
1008924,4,@FlashDenNet LOL TGIF is not true for freelanc...,@FlashDenNet LOL TGIF is not true for freelanc...,@FlashDenNet LOL TGIF is not true for freelanc...,lol tgif is not true for freelancers,"[lol, tgif, not, true, freelancers]"
599644,0,@Shayini As am I,@Shayini As am I,@Shayini As am I,as am i,[]
513071,0,where is the sun???? I hate rain..especially w...,where is the sun???? I hate rain..especially w...,where is the sun???? I hate rain..especially w...,where is the sun i hate rainespecially when i ...,"[sun, hate, rainespecially, walk, dog]"


In [21]:
#Check for duplicates and nulls again if occurs
data.isnull().sum()
data.duplicated(subset=['polarity', 'text','emoji_text','expanded_text','clean text']).sum()

np.int64(0)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1517962 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   polarity       1517962 non-null  int64 
 1   text           1517962 non-null  object
 2   emoji_text     1517962 non-null  object
 3   expanded_text  1517962 non-null  object
 4   clean text     1517962 non-null  object
 5   tokens         1517962 non-null  object
dtypes: int64(1), object(5)
memory usage: 81.1+ MB


In [23]:
#Lets Lemmatize the token list
lemmatizer = WordNetLemmatizer()
data['tokens']=data['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
data.sample(10)

,polarity,text,emoji_text,expanded_text,clean text,tokens
1369317,4,@SusanRaymond I'll pay you cash money to come ...,@SusanRaymond I'll pay you cash money to come ...,@SusanRaymond I will pay you cash money to com...,i will pay you cash money to come take the cat...,"[pay, cash, money, come, take, cat, come, arou..."
1330237,4,@ancmowat ...a good artist friend of my recogn...,@ancmowat ...a good artist friend of my recogn...,@ancmowat ...a good artist friend of my recogn...,a good artist friend of my recognized me and i...,"[good, artist, friend, recognized, like, share..."
1414771,4,@theaofa Just buy the shoes and hide them in t...,@theaofa Just buy the shoes and hide them in t...,@theaofa Just buy the shoes and hide them in t...,just buy the shoes and hide them in the car an...,"[buy, shoe, hide, car, bring, later, asks, say..."
582274,0,3.0 on iPhone still can't spell...,3.0 on iPhone still can't spell...,3.0 on iPhone still cannot spell...,on iphone still cannot spell,"[iphone, still, not, spell]"
149821,0,woke up feelin a lil sneezy this morning ughh...,woke up feelin a lil sneezy this morning ughh...,woke up feelin a lil sneezy this morning ughh...,woke up feelin a lil sneezy this morning ughh ...,"[woke, feelin, lil, sneezy, morning, ughh, no,..."
1422243,4,@Kissmekayley i wouldn't be standing if i'd ha...,@Kissmekayley i wouldn't be standing if i'd ha...,@Kissmekayley i would not be standing if i wou...,i would not be standing if i would had that ma...,"[would, not, standing, would, manybeen, long, ..."
1542571,4,@flourgrrrl love it. just joined. thanks! ton...,@flourgrrrl love it. just joined. thanks! ton...,@flourgrrrl love it. just joined. thanks! ton...,love it just joined thanks tonight i am planni...,"[love, joined, thanks, tonight, planning, make..."
1039459,4,@kirstiealley @MarleeMatlin Just got Tweet fro...,@kirstiealley @MarleeMatlin Just got Tweet fro...,@kirstiealley @MarleeMatlin Just got Tweet fro...,just got tweet from marlee pretending i cannot...,"[got, tweet, marlee, pretending, not, read, ca..."
193947,0,wow. The Goo Goo Dolls only have 3 tour dates ...,wow. The Goo Goo Dolls only have 3 tour dates ...,wow. The Goo Goo Dolls only have 3 tour dates ...,wow the goo goo dolls only have tour dates pos...,"[wow, goo, goo, doll, tour, date, posted, webs..."
36211,0,"sacrifice, fulfillment and gratitude..my heart...","sacrifice, fulfillment and gratitude..my heart...","sacrifice, fulfillment and gratitude..my heart...",sacrifice fulfillment and gratitudemy heart it...,"[sacrifice, fulfillment, gratitudemy, heart, h..."


In [26]:
tokenized_texts = data['tokens'].apply(lambda x: ' '.join(x)).tolist()
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(tokenized_texts)

sequences = tokenizer.texts_to_sequences(tokenized_texts)
max_length = 50
x = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

In [ ]:
x

array([[ 1621,  1041,  3075, ...,     0,     0,     0],
       [  620,     1,   274, ...,     0,     0,     0],
       [  200,    13,   814, ...,     0,     0,     0],
       ...,
       [  118,  6873,  6657, ...,     0,     0,     0],
       [   56,   192,   134, ...,     0,     0,     0],
       [   56, 15706,     0, ...,     0,     0,     0]], dtype=int32)

In [28]:
y = data['polarity'].replace({4: 1}).values
y = y.astype(np.int32)
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [30]:
#Save the feature data and target, also save the tokenizer so later you can map new data also to the trained model.
np.save('/content/drive/MyDrive/Colab Notebooks/Sentiment Analysis/features.np',x)
np.save('/content/drive/MyDrive/Colab Notebooks/Sentiment Analysis/labels.np',y)
with open('/content/drive/MyDrive/Colab Notebooks/Sentiment Analysis/tokenizer.keras','wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)